In [1]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [3]:
%run -i ../../zkstats/core.py

In [4]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')

=======================  ZK-STATS FLOW =======================

In [5]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

data = json.loads(open(data_path, "r").read())['col_name']
data_tensor = torch.reshape(torch.tensor(data),(1,-1, 1))

createDummy(data_path, dummy_data_path)
dummy_data = json.loads(open(dummy_data_path, "r").read())['col_name']
dummy_data_tensor = torch.reshape(torch.tensor(dummy_data), (1,-1,1))

dummy_theory_output = torch.sqrt(torch.var(dummy_data_tensor, correction = 0))
dummy_data_mean = torch.mean(dummy_data_tensor)
theory_output = torch.sqrt(torch.var(data_tensor, correction = 0))
data_mean = torch.mean(data_tensor)

In [6]:
# Verifier/ data consumer side:
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        self.w = nn.Parameter(data = dummy_theory_output, requires_grad = False)
        self.data_mean = nn.Parameter(data = dummy_data_mean, requires_grad = False)

    def forward(self,X):
        x_mean_cons = torch.abs(torch.sum(X)-X.size()[1]*(self.data_mean))<=torch.abs(0.01*X.size()[1]*self.data_mean)
        # since we square
        return (torch.logical_and(torch.abs(torch.sum((X-self.data_mean)*(X-self.data_mean))-self.w*self.w*X.size()[1])<=torch.abs(0.02*self.w*self.w*X.size()[1]),x_mean_cons ),self.w)

        
verifier_define_calculation(dummy_data_path, ['col_name'],sel_dummy_data_path,verifier_model, verifier_model_path)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/onnx/symbolic_opset9.py:2174: FutureWarning: 'torch.onnx.symbolic_opset9._cast_Bool' is deprecated in version 2.0 and will be removed in the future. Please Avoid using this function and create a Cast node instead.
  return fn(g, to_cast_func(g, input, False), to_cast_func(g, other, False))


In [7]:
# Prover/ data owner side
print("theory output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        self.w = nn.Parameter(data = theory_output, requires_grad = False)
        self.data_mean = nn.Parameter(data = data_mean, requires_grad = False)
    def forward(self,X):
        x_mean_cons = torch.abs(torch.sum(X)-X.size()[1]*(self.data_mean))<=torch.abs(0.01*X.size()[1]*self.data_mean)
        # since we square
        return (torch.logical_and(torch.abs(torch.sum((X-self.data_mean)*(X-self.data_mean))-self.w*self.w*X.size()[1])<=torch.abs(0.02*self.w*self.w*X.size()[1]),x_mean_cons ),self.w)

prover_gen_settings(data_path,['col_name'], sel_data_path, prover_model,prover_model_path, [2], "resources", settings_path)

theory output:  tensor(14.5557)
==== Generate & Calibrate Setting ====
scale:  [2]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":2,"param_scale":2,"scale_rebase_multiplier":10,"lookup_range":[-40670,8],"logrows":16,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":1521,"total_const_size":8,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,2],"model_input_scales":[2],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"GreaterThan":{"a":0.0}}],"check_mode":"UNSAFE","version":"7.0.0","num_blinding_factors":null}


In [8]:
# Here verifier & prover can concurrently call setup since all params are public to get pk. 
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

spawning module 0
spawning module 2


==== setting up ezkl ====


spawning module 0
spawning module 2


Time setup: 6.110619068145752 seconds
Theory output:  tensor(14.5557)
!@# compiled_model exists? True
!@# compiled_model exists? True
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 14.5
==== Generating Proof ====


spawning module 0
spawning module 2


proof:  {'instances': [[[12907834141446617622, 16863260785082668294, 2215826187815521673, 2191547160659437828], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [15846044116984004302, 17434658319873045084, 12685703199754313893, 2889686633444970017]]], 'proof': '01578017fe50c728797545bf25b4f2c7fbe051d16c5a0a555d8631f3238bc7e424e850aebe9e29dec8644bb540f95182bdc0eafe80511875a451c359cfebe78c27320798e152607ed040f0dcc27d2e7e987e78d6fd7a82aa8fceda0f49d08dbb170a6d35dd0440a1031b794bcb05b39311c32060a21406e131dc6fc91e58fa4730394a5d81310a89ee9af906e741f428d85a7b8606b7e7fd087817d14e804efc06306ffc5f739e6ee0a0f437cd33d512e1b1ccfcb320ca9605a63230224cb83d2695aa1bea3fbcf7ddcba4081dd0a5bab5fa07da31cc8797534e1d00964ad14627d7db7342e080cc2e618c563d350f4f36c2a450859bf6ef8f78aec5d57a951619ab87e7cfae8078c143c70d7771cf629edd394c6931c192601c27404a25aac90fcd324245ba62ed288b5e4909315eb3a2274ed99dfd9439c7be788941f8b66a266f6a8935e452bc6cdb60c096193b0ed6f4a2764c33d1238e51a6e69924

In [10]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path)

num_inputs:  1
prf instances:  [[[12907834141446617622, 16863260785082668294, 2215826187815521673, 2191547160659437828], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [15846044116984004302, 17434658319873045084, 12685703199754313893, 2889686633444970017]]]
proof boolean:  1.0
proof result 1 : 14.5
verified
